In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import joblib

# Charger les données
df_vecteur = pd.read_csv('../../data/processed/clean_dataset.csv')

# Séparer les caractéristiques et la cible
X = df_vecteur['merged']
y = df_vecteur["prdtypecode"]

# Encoder la cible 
label_encoder = joblib.load('label_encoder.joblib')
y_encoded = label_encoder.transform(y)

# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Définir le pipeline pour l'entraînement
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_.astype(str)))

# Extraire les objets entraînés (sans SMOTE)
tfidf_trained = pipeline.named_steps['tfidf']
xgb_trained = pipeline.named_steps['xgb']

# Définir le pipeline pour l'inférence (ie sans SMOTE) et avec sklearn.pipeline pour la compatibilité avec le VotingClassifier
from sklearn.pipeline import Pipeline as SklearnPipeline

inference_pipeline = SklearnPipeline([
    ('tfidf', tfidf_trained),
    ('xgb', xgb_trained)
])

# Sauvegarde du pipeline complet
joblib.dump(inference_pipeline, 'inference_pipeline_xgb_model.joblib')

# Sauvegarde du label encoder
#joblib.dump(label_encoder, 'label_encoder.joblib')


/home/karim/SEP24_CDS_Rakuten/env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [23:10:48] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

          10       0.20      0.61      0.31       397
          40       0.52      0.44      0.47       295
          50       0.71      0.64      0.67       225
          60       0.90      0.90      0.90       103
        1140       0.61      0.60      0.60       309
        1160       0.76      0.54      0.63       487
        1180       0.24      0.23      0.24       108
        1280       0.66      0.50      0.57       669
        1281       0.53      0.48      0.50       282
        1300       0.81      0.78      0.79       670
        1301       0.53      0.63      0.57        79
        1302       0.77      0.67      0.72       299
        1320       0.63      0.59      0.61       391
        1560       0.75      0.74      0.74       641
        1920       0.85      0.85      0.85       528
        1940       0.62      0.60      0.61       122
        2060       0.75      0.73      0.74       681
        2220       0.75    

['inference_pipeline_xgb_model.joblib']